In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

%matplotlib inline

In [ ]:
df_stores = pd.read_csv('../input/store.csv', sep=',')
df_data = pd.read_csv('../input/train.csv', sep=',')
df_test = pd.read_csv('../input/test.csv', sep=',')

In [ ]:
print(df_stores.shape)
print(df_data.shape)
print(df_test.shape)

In [ ]:
#Для первого приближения решил выкинуть ряд признаков
df_stores = df_stores.drop('CompetitionOpenSinceMonth', axis=1).drop('CompetitionOpenSinceYear', axis=1).drop('Promo2SinceWeek', axis=1).drop('Promo2SinceYear', axis=1)

In [ ]:
df_stores.head()

In [ ]:
df_data.head()

In [ ]:
df_test.head()

In [ ]:
#Удаляем все строки, когда магазины закрыты
closed_stores = df_test["Id"][df_test["Open"] == 0].values
df_test = df_test[df_test["Open"] != 0]
df_data = df_data[df_data["Open"] != 0]

In [ ]:
#Число покупателей тоже решил удалить, поскольку в тесте их нет
df_data = df_data.drop('Open', axis=1).drop('Customers', axis=1)
df_test = df_test.drop('Open', axis=1)

In [ ]:
#Дату заменил на месяц и год
df_data['Month'] = df_data['Date'].apply(lambda x: int(x[5:7]))
df_test['Month'] = df_test['Date'].apply(lambda x: int(x[5:7]))
df_data['Year'] = df_data['Date'].apply(lambda x: int(x[:4]))
df_test['Year'] = df_test['Date'].apply(lambda x: int(x[:4]))
df_test = df_test.drop('Date', axis=1)
df_data = df_data.drop('Date', axis=1)

In [ ]:
#Показалось целесообразным ввести для магазинов дополнительную категорию - средние продажи,
#поскольку может быть сильная зависимость от города и т. д., которую можно учесть.
df_stores = df_stores.join(df_data.groupby('Store')['Sales'].mean(), on='Store').rename(columns={'Sales': 'Av_sales'})

In [ ]:
df_stores.head()

In [ ]:
#Нормировка на единицу
max_s = df_stores['Av_sales'].max()
df_stores['Av_sales']=df_stores['Av_sales']/max_s

In [ ]:
df_test = pd.merge(df_test, df_stores, left_index=True, on='Store')
df_data = pd.merge(df_data, df_stores, left_index=True, on='Store')

In [ ]:
df_data.head()

In [ ]:
df_data['CompetitionDistance'].fillna(df_data['CompetitionDistance'].median(), inplace = True)
df_test['CompetitionDistance'].fillna(df_data['CompetitionDistance'].median(), inplace = True)

In [ ]:
max_dist = df_data['CompetitionDistance'].max()
df_data['CompetitionDistance']=df_data['CompetitionDistance']/max_dist
df_test['CompetitionDistance']=df_test['CompetitionDistance']/max_dist

In [ ]:
df_data['PromoInterval'].fillna('n', inplace = True)
df_test['PromoInterval'].fillna('n', inplace = True)

In [ ]:
#Для всех категориальных признаков, которые не 0 и 1, посмотрел, 
#насколько в среднем они влияют на продажи и заменил на пропорциональные числовые значения
print('Days', df_data['DayOfWeek'].unique())
print ('Month', df_data['Month'].unique())
print ('Promo', df_data['Promo'].unique())
print ('StateHoliday', df_data['StateHoliday'].unique())
print ('SchoolHoliday', df_data['SchoolHoliday'].unique())
print ('StoreType', df_data['StoreType'].unique())
print ('Assortment', df_data['Assortment'].unique())
print ('PromoInterval', df_data['PromoInterval'].unique())
print ('Year', df_data['Year'].unique())

In [ ]:
df_data["StateHoliday"].loc[df_data["StateHoliday"] == 0] = "0"
df_test["StateHoliday"].loc[df_test["StateHoliday"] == 0] = "0"

In [ ]:
print('StateHoliday', df_data['StateHoliday'].unique())

In [ ]:
df_test.isnull().any()

In [ ]:

features = ['DayOfWeek', 'Month', 'Promo', 'StateHoliday', 'SchoolHoliday',
            'StoreType', 'Assortment', 'PromoInterval', 'Year']

for fea in features:
    av_sales = df_data.groupby(fea)['Sales'].mean().values
    num_dict = dict(df_data.groupby(fea)['Sales'].mean()/max(av_sales))
    df_data[fea] = df_data[fea].map(num_dict)
    df_test[fea] = df_test[fea].map(num_dict)
    print(fea, num_dict, '\n')

In [ ]:
df_data.head()

In [ ]:
#Удалил из данных некие выбросы
df_data = df_data[df_data['Sales']!=0]

In [ ]:
hist, bins = np.histogram(df_data['Sales'].values, bins=100)

In [ ]:
bins

In [ ]:
hist

In [ ]:
df_data = df_data[df_data['Sales']<34500]

In [ ]:
df_data.shape

In [ ]:
df_data = df_data.drop('Store', axis=1)
df_test = df_test.drop('Store', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_mytest = train_test_split(df_data, test_size = 0.2)

In [ ]:
X_train, y_train = df_train.drop('Sales', axis=1).values, df_train['Sales'].values

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(n_jobs=4)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
weights = model.coef_
cols = df_train.drop('Sales', axis=1).columns.values
for i in range(len(cols)):
    print(weights[i], cols[i])

In [ ]:
def eval_rmspe(y_hat, y_true):
    res = np.sqrt(sum(((y_true-y_hat)/y_true)**2)/len(y_true))
    return res

In [ ]:
X_test, y_mytest = df_mytest.drop('Sales', axis=1).values, df_mytest['Sales'].values

In [ ]:
y_hat = model.predict(X_test)

In [ ]:
print(eval_rmspe(y_hat, y_mytest))

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
R_model = Ridge(alpha=1)
R_model.fit(X_train, y_train)

In [ ]:
y_hat_R = R_model.predict(X_test)
print(eval_rmspe(y_hat_R, y_mytest))

In [ ]:
#Посмотрим, как работает случайный лес
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#forest = RandomForestRegressor(n_jobs=4, n_estimators=100, max_features=8, max_depth=100)

In [ ]:
#forest.fit(X_train, y_train)

In [ ]:
#y_hat_forest = forest.predict(X_test)

In [ ]:
#print(eval_rmspe(y_hat_forest, y_mytest))

In [ ]:
#Случайный лес обучается довольно долго, поэтому к подбору гиперпараметров вернусь позже
submission = pd.Series()
submission = submission.append(pd.Series(0, index=closed_stores))
submission.head()

In [ ]:
#Берём полные данные для обучения
X_train_full, y_train_full = df_data.drop('Sales', axis=1).values, df_data['Sales'].values

In [ ]:
X_test_full = df_test.drop('Id', axis=1).values

In [ ]:
#Память переполняется, поэтому обучение идёт не очень большим количеством деревьев
forest = RandomForestRegressor(n_jobs=1, n_estimators=150, max_features=7, max_depth=100)
forest.fit(X_train_full, y_train_full)

In [ ]:
y_hat_full = forest.predict(X_test_full)

In [ ]:
store_ids = df_test['Id'].values
store_ids

In [ ]:
submission = submission.append(pd.Series(y_hat_full, index=store_ids))
submission = pd.DataFrame({ "Id": submission.index, "Sales": submission.values})
submission.to_csv('submission.csv', index=False)

In [ ]:
#Private Score 0.14253
#Public Score 0.12313

In [ ]:
#Private Score 0.14253
#Public Score 0.12313

In [ ]:
print(check_output(["ls", "./"]).decode("utf8"))